In [0]:
%run "../includes/configuration"

In [0]:
driver_df = spark.read.parquet(f"{processed_folder_path}/driver").withColumnRenamed("number", "driver_number").withColumnRenamed("name", "driver_name").withColumnRenamed("nationality", "driver_nationality")

In [0]:
constructor_df = spark.read.parquet(f"{processed_folder_path}/constructor").withColumnRenamed("name", "team").withColumnRenamed("constructorId", "constructor_id")

In [0]:
circuit_df  = spark.read.parquet(f"{processed_folder_path}/circuits").withColumnRenamed("location", "circuit_location")

In [0]:
races_df = spark.read.parquet(f"{processed_folder_path}/races").withColumnRenamed("name", "race_name").withColumnRenamed("race_timestamp", "race_date")

In [0]:
result_df = spark.read.parquet(f"{processed_folder_path}/results").withColumnRenamed("time", "race_time")

In [0]:
race_circuit_df = races_df.join(circuit_df, races_df.circuit_id == circuit_df.circuit_id).select(races_df.race_id, races_df.race_year, races_df.race_name, races_df.race_date, circuit_df.circuit_location)

In [0]:
race_result_df = result_df.join(race_circuit_df, result_df.race_id == race_circuit_df.race_id).join(driver_df, result_df.driver_id == driver_df.driver_id).join(constructor_df, result_df.constructor_id == constructor_df.constructor_id)

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = race_result_df.select("race_year", "race_name", "race_date", "circuit_location", "driver_name", "driver_number", "driver_nationality", "team", "grid", "fastest_lap", "race_time", "points", "position")

In [0]:
display(final_df.filter("race_year == 2020 and race_name == 'Abu Dhabi Grand Prix'").orderBy(final_df.points.desc()))

race_year,race_name,race_date,circuit_location,driver_name,driver_number,driver_nationality,team,grid,fastest_lap,race_time,points,position
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Max Verstappen,33,Dutch,Red Bull,1,14,1:36:28.645,25.0,1
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Valtteri Bottas,77,Finnish,Mercedes,2,40,+15.976,18.0,2
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Lewis Hamilton,44,British,Mercedes,3,37,+18.415,15.0,3
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Alexander Albon,23,Thai,Red Bull,5,42,+19.987,12.0,4
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Lando Norris,4,British,McLaren,4,53,+1:00.729,10.0,5
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Carlos Sainz,55,Spanish,McLaren,6,48,+1:05.662,8.0,6
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Daniel Ricciardo,3,Australian,Renault,11,55,+1:13.748,7.0,7
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Pierre Gasly,10,French,AlphaTauri,9,53,+1:29.718,4.0,8
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Esteban Ocon,31,French,Renault,10,47,+1:41.069,2.0,9
2020,Abu Dhabi Grand Prix,2020-12-13T13:10:00.000+0000,Abu Dhabi,Lance Stroll,18,Canadian,Racing Point,8,41,+1:42.738,1.0,10


In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.race_results")